In [1]:
import pandas as pd
import sys
import requests as r

sys.path.append('../helper_functions')
import github_utils as gh
import tokenlist_utils as tl
import duneapi_utils as dapi

In [2]:
gh_url = gh.get_file_url_from_github('ethereum-optimism','ethereum-optimism.github.io','','optimism.tokenlist.json')
dune_list_query_id = 2420477 #https://dune.com/queries/2420477

In [3]:
# Load the JSON data from the URL
res = r.get(gh_url)
data = res.json()


df = tl.generate_table_from_tokenlist(data)
df = df[df['chainId'] ==10] #Get OP Version
display(df)

,chainId,address,name,symbol,decimals,extensions_optimismBridgeAddress,extensions_opListId,extensions_opTokenId
1,10,0xe0BB0D3DE8c10976511e5030cA403dBf4c25165B,0xBitcoin,0xBTC,8,0x4200000000000000000000000000000000000010,extended,0xBTC
3,10,0x76FB31fb4af56892A25e32cFC43De717950c9278,Aave Token,AAVE,18,0x4200000000000000000000000000000000000010,default,AAVE
5,10,0x764ad60e1b81f6cacfec1a2926393d688d4493e6,AladdinCRV,aCRV,18,0x4200000000000000000000000000000000000010,extended,aCRV
7,10,0xff733b2a3557a7ed6697007ab5d11b79fdd1b76b,Across Protocol Token,ACX,18,0x4200000000000000000000000000000000000010,extended,ACX
8,10,0x61BAADcF22d2565B0F471b291C475db5555e0b76,Aelin Protocol,AELIN,18,NaN,extended,AELIN
...,...,...,...,...,...,...,...,...
252,10,0x1F32b1c2345538c0c6f582fCB022739c4A194Ebb,Wrapped liquid staked Ether 2.0,wstETH,18,0x8E01013243a96601a86eb3153F0d9Fa4fbFb6957,default,wstETH
254,10,0xdb4eA87fF83eB1c80b8976FC47731Da6a31D35e5,wTBT(Bridge Token),wTBT,18,0x4200000000000000000000000000000000000010,extended,wTBT
256,10,0xE4F27b04cC7729901876B44f4EAA5102EC150265,CryptoFranc,XCHF,18,0x4200000000000000000000000000000000000010,extended,XCHF
258,10,0xD1917629B3E6A72E6772Aab5dBe58Eb7FA3C2F33,0x Protocol Token,ZRX,18,0x4200000000000000000000000000000000000010,default,ZRX


In [4]:
dune_list = dapi.get_dune_data(dune_list_query_id)
display(dune_list)

2023-05-03 16:26:28.711 | INFO     | duneapi_utils:get_dune_data:25 - Results available at https://dune.com/queries/2420477
2023-05-03 16:26:29,790 INFO dune_client.base_client waiting for query execution 01GZHMQ0CFXJYFWK4DEN4PDK8M to complete: ExecutionState.EXECUTING
2023-05-03 16:26:36.109 | INFO     | duneapi_utils:get_dune_data:40 - ✨ Results saved as csv_outputs/my_query_results.csv, with 372 rows and 7 columns.


,contract_address,decimals,is_counted_in_tvl,symbol,token_mapping_source,token_type,last_updated
0,0x2e17b8193566345a2dd467183526dedc42d2d5a8,18,0,HOP-LP-USDC,manual,receipt,2023-05-03 20:26:29.136531+00:00
1,0x25d8039bb044dc227f741a9e381ca4ceae2e6ae8,6,1,hUSDC,manual,underlying,2023-05-03 20:26:29.136531+00:00
2,0xc40f949f8a4e094d1b49a23ea9241d289b7b2819,18,1,LUSD,manual,underlying,2023-05-03 20:26:29.136531+00:00
3,0xb344795f0e7cf65a55cb0dde1e866d46041a2cc2,6,0,iUSDC,manual,receipt,2023-05-03 20:26:29.136531+00:00
4,0x94b008aa00579c1307b0ef2c499ad98a8ce58e58,6,1,USDT,manual,underlying,2023-05-03 20:26:29.136531+00:00
...,...,...,...,...,...,...,...
367,0x588abc030b08819c4c284189ce269a8fb4efe439,18,1,quotMKRquot,manual,underlying,2023-05-03 20:26:29.136531+00:00
368,0x39fde572a18448f8139b7788099f0a0740f51205,18,1,OATH,manual,underlying,2023-05-03 20:26:29.136531+00:00
369,0xcdb4bb51801a1f399d4402c61bc098a72c382e65,18,1,OPX,manual,underlying,2023-05-03 20:26:29.136531+00:00
370,0xe50fa9b3c56ffb159cb0fca61f5c9d750e8128c8,18,0,aOptWETH,manual,receipt,2023-05-03 20:26:29.136531+00:00


In [5]:
#token addresses to lower
df['contract_address'] = df['address'].str.lower()
dune_list['contract_address'] = dune_list['contract_address'].str.lower()

In [6]:
adds = df[~df['contract_address'].isin(dune_list['contract_address'])]
display(adds[['address','symbol','decimals']])


,address,symbol,decimals
5,0x764ad60e1b81f6cacfec1a2926393d688d4493e6,aCRV,18
18,0xDb9888b842408B0b8eFa1f5477bD9F351754999E,BAXA,18
33,0xCa0E54b636DB823847B29F506BFFEE743F57729D,CHI,18
64,0x7b0bcC23851bBF7601efC9E9FE532Bf5284F65d3,EST,18
120,0x3bB4445D30AC020a84c1b5A8A2C6248ebC9779D0,LIZ,18
122,0x0B3e851cf6508A16266BC68A651ea68b31ef673b,LPF,18
134,0x77D40CBc27f912dcDbF4348cAf87B427c4D02486,MOCHI,18
138,0x5e70AfFE232e2919792f77EB94e566db0320fa88,MOM,18
233,0x678d8f4Ba8DFE6bad51796351824DcCECeAefF2B,veKWENTA,18
260,0x6e4cc0Ab2B4d2eDAfa6723cFA1582229F1Dd1Be1,ZUSD,6
